In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import gc
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
import seaborn as sns
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
import catboost as catt
import xgboost as xgb
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/umojahack-cryptojacking/SampleSubmission.csv
/kaggle/input/umojahack-cryptojacking/Cryptojacking_Detection_Starter_Notebook.ipynb
/kaggle/input/umojahack-cryptojacking/Train.csv
/kaggle/input/umojahack-cryptojacking/Test.csv
/kaggle/input/umojahack-cryptojacking/Variable_Definitions.csv


In [2]:
#import datasets
train = pd.read_csv("../input/umojahack-cryptojacking/Train.csv")
test = pd.read_csv("../input/umojahack-cryptojacking/Test.csv")
SampleSubmission = pd.read_csv("../input/umojahack-cryptojacking/SampleSubmission.csv")

In [3]:
ntrain= train.shape[0]
ntest = test.shape[0]
train_set = pd.concat((train, test)).reset_index(drop=True)

In [4]:
train.head()

,ID,I/O Data Operations,I/O Data Bytes,Number of subprocesses,Time on processor,Disk Reading/sec,Disc Writing/sec,Bytes Sent/sent,Received Bytes (HTTP),Network packets sent,Network packets received,Pages Read/sec,Pages Input/sec,Page Errors/sec,Confirmed byte radius,Label
0,ID_4W8AP96UO6,114.798900,3790.450939,28.0,0.427078,6.162365,21.220403,58.492773,63.959387,0.621206,0.521813,24.624612,0.0,1001.533430,27.190843,1
1,ID_UD3TM0ZYND,14.154240,5182.451722,28.0,0.138876,5.399498,0.000000,22.886762,29.508369,0.422183,0.444403,0.044440,0.0,1089.121009,30.150941,0
2,ID_XAG1HC0HWM,0.044444,120.821253,28.0,0.000000,1.577765,0.000000,11.733239,12.888786,0.177776,0.155554,0.044444,0.0,322.464081,28.163287,0
3,ID_CEXD05IR09,31.369916,127228.251100,31.0,0.523750,15.981107,5.356671,602.125436,6976.986795,3.311801,5.756755,5.623394,0.0,12015.169690,34.204404,0
4,ID_X6E97FT8IF,5.046181,1091.620117,25.0,0.069468,1.356022,0.000000,16.138887,13.271234,0.155609,0.133379,0.000000,0.0,550.989619,31.466889,1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8908 entries, 0 to 8907
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        8908 non-null   object 
 1   I/O Data Operations       8908 non-null   float64
 2    I/O Data Bytes           8908 non-null   float64
 3   Number of subprocesses    8908 non-null   float64
 4   Time on processor         8908 non-null   float64
 5   Disk Reading/sec          8908 non-null   float64
 6   Disc Writing/sec          8908 non-null   float64
 7   Bytes Sent/sent           8908 non-null   float64
 8   Received Bytes (HTTP)     8908 non-null   float64
 9   Network packets sent      8908 non-null   float64
 10  Network packets received  8908 non-null   float64
 11  Pages Read/sec            8908 non-null   float64
 12  Pages Input/sec           8908 non-null   float64
 13  Page Errors/sec           8908 non-null   float64
 14  Confirme

In [6]:
train.shape

(8908, 16)

In [7]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
I/O Data Operations,8908.0,36.429205,75.884647,0.000000,1.394404,11.613803,34.494953,1.795316e+03
I/O Data Bytes,8908.0,113556.435854,489684.616342,0.000000,813.631825,4930.481627,39403.570772,4.677280e+06
Number of subprocesses,8908.0,29.811900,5.691411,1.000000,27.000000,28.000000,30.000000,5.100000e+01
Time on processor,8908.0,0.496405,1.593626,0.000000,0.000000,0.034734,0.451350,2.524615e+01
Disk Reading/sec,8908.0,5.353260,19.286690,0.044459,0.755629,1.711077,5.845619,1.064819e+03
Disc Writing/sec,8908.0,1.046153,15.667814,0.000000,0.000000,0.000000,0.022229,8.315039e+02
Bytes Sent/sent,8908.0,559.546628,1939.745234,1.197890,33.843800,67.366295,221.314810,9.809778e+04
Received Bytes (HTTP),8908.0,16558.044068,90428.025210,1.554938,27.377705,89.057696,561.381397,4.812144e+06
Network packets sent,8908.0,4.795485,20.797585,0.022183,0.399999,0.710898,1.555063,1.118878e+03
Network packets received,8908.0,12.625792,61.246541,0.022183,0.333225,0.635705,1.711155,3.183701e+03
